In [122]:
from bs4 import BeautifulSoup
import requests
import os
from io import open

In [123]:
periodico="https://www.20minutos.es"
tipo_periodico="20minutos" 
counter_dictionary={}

In [124]:
def veinteminsPalabrasClaves(soup):
    palabras_claves = ""
    separador = ", "
    # lista con todos los palabras claves
    html_tags = soup.find_all("li", class_="tag")
    # solo las palabras que son enlaces
    # Sacamos el texto de los enlaces y lo añadimos a la lista 
    for tag in html_tags:
        palabras_claves = palabras_claves + tag.find("a").text.strip() + separador
    # Quitamos el ultimo separador
    palabras_claves = palabras_claves[:-(len(separador))]
    return palabras_claves

def veinteminsParrafos(soup):
    content = ""
    #buscamos el div de la class donde se encuentra todo el cuerpo del artículo
    text = soup.find("div", class_="article-text")
    # buscar todos los p
    paragraphs = text.find_all("p")
    # sacar el texto del p y anadir al content
    for paragraph in paragraphs:
        content = content + paragraph.text + "\n"
    return content

def veinteminsTitulo(soup):
    return soup.find("h1", class_="article-title").text

def veinteminsIntro(soup):
    return soup.find("li", class_="article-intro").text

def veinteminsFecha(soup):
    date_length = 10 # 2021-12-03T18:09:39.529Z
    return soup.find("meta", property="article:published_time")["content"][:date_length]

def guardarFichero(contenido, fecha, tipo_periodico, categoria):
    # calcular numero (nnn)
    counter = 1
    if fecha in counter_dictionary[categoria]:
        counter = counter_dictionary[categoria][fecha]
        counter_dictionary[categoria][fecha] += 1
    else:
        counter_dictionary[categoria][fecha] = 2
    
    # /diario/categoria/categoria.fecha.nnn.txt
    abs_path = os.getcwd()
    rel_path =  tipo_periodico + "/" + categoria + "/"
    nombre_fichero = categoria + "." + fecha + "." + str(counter).zfill(3) + ".txt"
    path = os.path.join(abs_path, rel_path, nombre_fichero)
    try:
        with open(path, 'x') as fichero:
            fichero.write(contenido)
        print ("File " + nombre_fichero + " created!")
    except IOError:
        print("File " + nombre_fichero + " already exists...")

In [125]:
def veinteminsWebScraping(pagina, tipo_periodico, categoria):
    soup = BeautifulSoup(pagina, 'html.parser')

    palabras_claves = veinteminsPalabrasClaves(soup)
    titulo = veinteminsTitulo(soup)
    entradilla = ""
    texto = veinteminsParrafos(soup)
    fecha = veinteminsFecha(soup)

    content = palabras_claves + "\n\n" + titulo + "\n\n" + entradilla + "\n\n" + texto
    guardarFichero(content, fecha, tipo_periodico, categoria)
    return content

In [126]:
def veintemins(periodico, tipo_periodico, categoria, url):
    counter_dictionary[categoria] = {}
    pagina = requests.get(url)
    soup = BeautifulSoup(pagina.content, 'html.parser')

    # sacar todas los enlaces de los articulos de la pagina categorial
    enlaces = []
    for article in soup.find_all("article"):
        enlaces.append(article.find("a")['href'])

    # processar cada articulo
    for enlace in enlaces: 
        articulo = requests.get(enlace)
        # veinteminsWebScraping(articulo. content, tipo_periodico, categoria)
        try:
            veinteminsWebScraping(articulo.content, tipo_periodico, categoria)
        except:
            print("Error procesando articulo, saltando el articulo " + enlace)

periodico="https://20minutos.es"
tipo_periodico="20minutos" 
counter_dictionary={}

# processar las categorias: salud, ciencia y tecnologia
categoria = "salud"
url ="https://20minutos.es/salud/"
veintemins(periodico, tipo_periodico, categoria, url)

categoria = "ciencia"
url = "https://www.20minutos.es/ciencia/"
veintemins(periodico, tipo_periodico, categoria, url)

categoria = "tecnologia"
url = "https://www.20minutos.es/tecnologia/"
veintemins(periodico, tipo_periodico, categoria, url)


File salud.2022-01-11.001.txt created!
File salud.2022-01-10.001.txt created!
File salud.2022-01-09.001.txt created!
File salud.2022-01-10.002.txt created!
File salud.2022-01-11.002.txt created!
File salud.2022-01-11.003.txt created!
File salud.2022-01-11.004.txt created!
File salud.2022-01-11.005.txt created!
File salud.2022-01-10.003.txt created!
File salud.2022-01-10.004.txt created!
File salud.2022-01-11.006.txt created!
File salud.2022-01-10.005.txt created!
File salud.2022-01-10.006.txt created!
File salud.2022-01-09.002.txt created!
File salud.2021-12-31.001.txt created!
File salud.2021-12-30.001.txt created!
File salud.2021-12-30.002.txt created!
File salud.2021-12-29.001.txt created!
File salud.2021-12-28.001.txt created!
File salud.2022-01-10.007.txt created!
File salud.2022-01-08.001.txt created!
File salud.2022-01-06.001.txt created!
File salud.2022-01-05.001.txt created!
Error procesando articulo, saltando el articulo https://www.20minutos.es/videos/salud/estos-son-los-eje